In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display

D = 0.190
H = 0.290
V = np.pi * D**2 * H / 4 # en m^3

# datos tomados

abs_blanc = 0.329
abs_tratada = 0.409
abs_residual = 0.426
Tiempo1 = 26.67 # en segundos
Volumen1 = 10 # en mL
Tiempo2 = 14
Volumen2 =  8
Tiempo3 =19
Volumen3 =11
Tiempo = (Tiempo1 + Tiempo2 + Tiempo3) / 3 # en segundos
Volumen = (Volumen1 + Volumen2 + Volumen3) / 3 # en mL

Q = Volumen / Tiempo # en mL/s
Q= Q/1000*3.6# en m³/h

DQO_blanc = abs_blanc / 4.27E-4
DQO_tratada = (abs_tratada-abs_blanc) / 4.27E-4
DQO_residual = (abs_residual-abs_blanc) / 4.27E-4

DQO_tabla = pd.DataFrame({'Absorbancia': [abs_blanc, abs_tratada, abs_residual],
                            'DQO': [None, DQO_tratada, DQO_residual]},
                             index=['Blanco', 'Tratada', 'Residual'])
display(DQO_tabla)

Rendimiento = (DQO_residual - DQO_tratada) / DQO_residual * 100 
Tresidencia = V / Q

tabla_rendimiento = pd.DataFrame({'caudal m³/h': [Q],
                                    'Rendimiento': [Rendimiento],
                                    'Tiempo de residencia': [Tresidencia]},
                                    index=['Datos'])
display(tabla_rendimiento)

T = 23 # temperatura de la habitacion
Q= Q*24 # m^3/dia
DQO_residual = int(DQO_residual)  # Ensure DQO_residual is an integer
DBO = DQO_residual * 0.67 # la cantidad de biomasa generada/cantidad no consumida xh/xs  o DBO/DQO es entre un 65% y 70%
k = 0.095 # d^-1  k oscila entre 0.7 y 0.12
DBOu = DBO * (1 - 10**(5 * k))
DQOb = DBOu * 1.14 
b = DBOu / DQOb
bh =  0.24 # cantidad heterotrofos que mueren/cantidad que habitan d^-1
u35 = 4 # tasa de crecimiento a 35 grados dias^-1
theta_b = 1.024 # constante de arrhenius para tasa de endogenesis
theta_u = 1.07 # constante de arrhenius para tasa de crecimiento
breal = bh*theta_b**(T-20) # theta b es la connstante de arrhenius para tasa de endogenesis
ureal = u35*theta_u**(T-35) # theta u es la constante de arrhenius para tasa de crecimiento
yh = 0.45 # g xh/g DQO
fH = 0.2 # (g DQO Xi/g DQOXh)
X0 = DQO_residual/1.42 # 160/113 [C5H7O2N + 5O2 -> 5CO2 + 3H2O + NH3]
S = DQO_residual - X0
Xs0 =  X0*b
XIo = X0 - Xs0  
S0 = DQO_residual - X0
ss0 = S0*b
Si0= ss0- S0
trs = Tresidencia
ss= 3000 # mg/L
mu= ureal*(S/k+S) # tasa de crecimiento microbiano ecuacion de monod
vxh = (((yh*(ss0-ss)*Q)/(1/trs + bh)))
delta_xh = vxh/trs # velocidad de degradacion
delta_xht = vxh*bh*fH
delta_xhi = XIo*Q/1.46
AX= delta_xh+delta_xht+delta_xhi
ssnv = ss*(1-0.8) # los solidos en suspension deberian ser de un 80% o superior pero supuesto 
fango_exceso = AX/1.42 + Q*ssnv # necesaria gravimetria para el ssnv

#constantes tomadas
tabla_constantes = pd.DataFrame({'k': [k],
                                 'bh': [bh],
                                 'u35': [u35],
                                 'yh': [yh],
                                 'fH': [fH],
                                 'theta_b': [theta_b],
                                 'theta_u': [theta_u],
                                 'trs': [trs],
                                 '%ssnv': [80]},  
                                index=['Datos'])
display(tabla_constantes)
tabla_calculos = pd.DataFrame({
    'Parametro': [
        'DBO', 'DBOu', 'DQOb', 'biodegradable', 'breal', 'ureal', 'X0', 'S', 'Xs0', 'XIo', 'S0', 'ss0', 'Si0', 'ss', 'mu', 'vxh', 'Qdelta_xh', 'Qdelta_xht', 'Qdelta_xhi', 'AX'
    ],
    'Resultado': [
        f"{DBO:.2f} mg O_2/L", f"{DBOu:.2f} mg O_2/L", f"{DQOb:.2f} mg O_2/L", f"{b:.2%}", f"{breal:.4f} d^-1", f"{ureal:.4f} d^-1", f"{X0:.2f} mg O_2/L", f"{S:.2f} mg/L", f"{Xs0:.2f}
          mg O_2/L", f"{XIo:.2f} mg O_2/L", f"{S0:.2f} mg O_2/L", f"{ss0:.2f} mg O_2/L", f"{Si0:.2f} mg O_2/L", f"{ss:.2f} mg/L", f"{mu:.4f} d^-1", f"{vxh:.2f} mg/L*d",
          f"{delta_xh:.2f} mg/dia", f"{delta_xht:.2f} mg/dia", f"{delta_xhi:.2f} mg/dia", f"{AX:.2f} mg/dia"
    ]
})
# fango en exceso
incxh = delta_xh/fango_exceso*100
incxht = delta_xht/fango_exceso*100
incxhi = delta_xhi/fango_exceso*100
incSSnv = Q*ssnv/fango_exceso*100
tabla_fangos = pd.DataFrame({
    'fango_exceso': [f"{fango_exceso:.2f}"],
    'incxh': [f"{incxh:.2f}%"],
    'incxht': [f"{incxht:.2f}%"],
    'incxhi': [f"{incxhi:.2f}%"],
    'incSSnv': [f"{incSSnv:.2f}%"],
})
# Mostrar la tabla
display(tabla_calculos)

,Absorbancia,DQO
Blanco,0.329,NaN
Tratada,0.409,187.353630
Residual,0.426,227.166276


,caudal m³/h,Rendimiento,Tiempo de residencia
Datos,0.00175,17.525773,4.699489


,k,bh,u35,yh,fH,theta_b,theta_u,trs,%ssnv
Datos,0.095,0.24,4,0.45,0.2,1.024,1.07,4.699489,80


,DBO,DBOu,DQOb,biodegradable,breal,ureal,X0,S,Xs0,XIo,S0,ss0,Si0,ss,mu,vxh,Qdelta_xh,Qdelta_xht,Qdelta_xhi,AX
0,152.09 mg O_2/L,-301.96 mg O_2/L,-344.23 mg O_2/L,87.72%,0.2577 d^-1,1.7760 d^-1,159.86 mg O_2/L,67.14 mg/L,140.23 mg O_2/L,19.63 mg O_2/L,67.14 mg O_2/L,58.90 mg O_2/L,-8.25 mg O_2/L,3000.00 mg/L,1374.4596 d^-1,-122.74 mg/L*d,-26.12 mg/dia,-5.89 mg/dia,0.56 mg/dia,-31.44 mg/dia
